In [11]:
import os
import pandas as pd

import custom_index as ci
import engine as en

In [2]:
DATA_DIR = os.path.join(os.getcwd(), 'temp_data')
print(DATA_DIR)

/Users/zhishengxie/PycharmProjects/personal_site_react/api/portfolio_construction_engine/temp_data


In [3]:
CUS_INDEX_CONFIG = os.path.join(DATA_DIR, 'custom_index_config.csv')

custom_index = ci.CustomIndex()
custom_index.load_data_from_csv(CUS_INDEX_CONFIG)
custom_index.fetch_ticker_data()
custom_index.construct_index()

Fetching data for 2 tickers from 2023-09-05 to 2023-10-01...
[*********************100%***********************]  2 of 2 completed


In [4]:
custom_index.returns

Index,index1
Date,
2023-09-06,-0.018901
2023-09-07,-0.019086
2023-09-08,0.008354
2023-09-11,0.008801
2023-09-12,-0.017659
2023-09-13,0.000538
2023-09-14,0.008319
2023-09-15,-0.014595
2023-09-18,0.006700


In [5]:
# Download factor data from yahoo finance
import yfinance as yf
def download_data(tickers: list, start_date, end_date, name: str):

    data = yf.download(tickers, start=start_date, end=end_date)
    data.index.name = 'DATE'
    data['Close'].to_csv(os.path.join(DATA_DIR, f'{name}.csv'))

    return data
NAME = 'factor_data'
START_DATE = '2022-01-01'
END_DATE = '2023-10-31'
tickers=['AAPL', 'MSFT']

# FACTOR_DATA = download_data(tickers, START_DATE, END_DATE, NAME)

In [6]:
PORTFOLIO = en.Portfolio(assets=['index1'], weights=[1])
PORTFOLIO.returns = custom_index.returns

In [7]:
FACTOR = en.Factor(os.path.join(DATA_DIR, f'{NAME}.csv'), convert_to_fraction=False)

In [22]:
RESULT = FACTOR.cal_custom_exposure(PORTFOLIO, factor_list=['AAPL', 'MSFT'], cal_excess_return=False)

In [33]:
# Extracting 'regression_stats' without using pop
regression_stats = RESULT['full_horizon']['regression_stats'].copy()
parameters = regression_stats['Parameters']

# Creating DataFrame for regression stats without removing 'Parameters'
df_regression_stats = pd.DataFrame([regression_stats])
df_regression_stats.drop(columns=['Parameters'], inplace=True)
df_regression_stats = df_regression_stats.T.reset_index().rename(columns={'index': 'Regression Stats', 0: 'Value'})
# Creating DataFrame for parameters
df_parameters = pd.DataFrame(parameters).T
df_parameters = df_parameters.reset_index().rename(columns={'index': 'Parameter'})


In [34]:
df_regression_stats

,Regression Stats,Value
0,R-squared,0.072161
1,F-statistic,0.583296
2,Autocorrelation,0.058275
3,Heteroscedasticity,0.377237
4,Time period,"[2023-09-06, 2023-09-29]"
5,Tickers,[index1]
6,Number of data points,18


In [35]:
df_parameters

,Parameter,coef,std_err,t_value,p_value,conf_int
0,const,0.029247,0.150069,0.194888,0.848096,"[-0.2906170108170015, 0.3491102442857295]"
1,AAPL,-0.001054,0.001215,-0.867596,0.399283,"[-0.003644753590709702, 0.001535967462604949]"
2,MSFT,0.000466,0.000436,1.069626,0.301699,"[-0.00046248059498098476, 0.0013942381086170126]"
